# Hydrography
-----------------
This notebook explores the hydrographic data from the glider, with a focus on asepcts that are germane to ADCP processing. These include:
- Dive average current
- Geostrophic current
- Sound speed estimates
- Depth of pycnocline and other interfaces commonly associated with velocity shear
- Water mass identification

In [ ]:
import sys
import numpy as np
import pandas as pd
import xarray as xr
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
from netCDF4 import Dataset
from pathlib import Path
import gsw
import scipy.io
from datetime import datetime, timedelta
notebook_dir = !pwd # Replace with !echo %cd% if using windows
library_dir = Path(notebook_dir[0]).parent.absolute()
sys.path.append(str(library_dir))
data_raw_path = library_dir / 'data' / 'raw' / 'adcp'
glider_raw_path = library_dir / 'data' / 'raw' / 'glider'
fig_path = library_dir / 'reports' / 'figures'
from src.data.adcp_mission_analysis import read_glider_nc, edgetocentre, glidertimetoneat

style_path = library_dir / 'src' / 'visualization' / 'presentation.mplstyle'
style.use(str(style_path))


def savefig(figname, extension="png"):
    plt.savefig(
        fig_path / str(figname + "." + extension),
        format=extension,
        dpi="figure",
        bbox_inches="tight",
    )



In [ ]:
glider_df = read_glider_nc(glider_raw_path / 'sg637_var_by_var_timeseries.nc')
#glider_df = read_glider_nc(glider_raw_path / 'sg637_flag_all_timeseries.nc')

Calculate sound speed

In [ ]:
glider_df['sound_speed'] = gsw.sound_speed(glider_df.abs_salinity, glider_df.temp, glider_df.pressure)

Propogate salinity flags across conductivity and density

In [ ]:
glider_df.salinity[glider_df.salinity<34.5] = np.nan
glider_df.salinity[glider_df.salinity>37.5] = np.nan
glider_df.abs_salinity[np.isnan(glider_df.salinity)] = np.nan
glider_df.conductivity[np.isnan(glider_df.salinity)] = np.nan
glider_df.rho[np.isnan(glider_df.salinity)] = np.nan
glider_df.sigma0[np.isnan(glider_df.salinity)] = np.nan
glider_df.sound_speed[np.isnan(glider_df.salinity)] = np.nan


In [ ]:
np.nansum(np.isnan(glider_df.abs_salinity.values))

In [ ]:
origin  = [-57.338, 14.182]
deg_lat = 110649
glider_df['x'] = (glider_df.lon - origin[0]) * deg_lat * np.cos(np.deg2rad(origin[1]))
glider_df['y'] = (glider_df.lat - origin[1]) * deg_lat

In [ ]:
glider_av_df = glider_df.groupby('dive', as_index=False).median()

In [ ]:
glider_av_df.describe()

In [ ]:
def glider_kongsberg_add(glider_df, kberg_dir):
    nc_paths = sorted(kberg_dir.rglob("p*.nc"))
    drift_u, drift_v, drift_e, dac_u, dac_v, dac_e = [], [], [], [], [], []
    for path in nc_paths:
        nc = xr.open_dataset(path)
        drift_u.append(nc.surface_curr_east.values)
        drift_v.append(nc.surface_curr_north.values)
        drift_e.append(nc.surface_curr_error.values)
        dac_u.append(nc.depth_avg_curr_east.values)
        dac_v.append(nc.depth_avg_curr_north.values)
        dac_e.append(nc.depth_avg_curr_error.values)
    for variable in [drift_u, drift_v, drift_e, dac_u, dac_v, dac_e]:
        glider_df[str(variable)] = variable
    return glider_df


In [ ]:
nc = xr.open_dataset( Path('/media/callum/storage/Documents/Eureka/data/glider/sg579/sg579_EUREC4A_timeseries.nc'))

In [ ]:
glider_av_df['glider_time'] = ''
noot = []
for dive in glider_av_df.dive:
    df = glider_df[glider_df.dive==dive]
    noot.append((df.glider_time - df.glider_time.min()).mean() + df.glider_time.min())
glider_av_df['glider_time'] = noot

In [ ]:
glider_av_df

In [ ]:
box_size = 1000
lon_edges = np.arange(-8000, 9000, box_size)
lat_edges = np.arange(-8000, 9000, box_size)
dives_in = np.empty((len(lon_edges), len(lat_edges)))
dives_in[:] = np.nan
for i in range(len(lon_edges[:-1])):
    glider_x_in = glider_av_df[np.logical_and(glider_av_df.x>lon_edges[i], glider_av_df.x<lon_edges[i+1])]
    for j in range(len(lat_edges[:-1])):
        glider_in = glider_x_in[np.logical_and(glider_x_in.y>lat_edges[j], glider_x_in.y<lat_edges[j+1])]
        dives_in[j,i] = len(glider_in)

### Geographical location of glider dives

Here we plot the locations of the glider dives. This is the average of glider lon and lat for each dive, converted to distance from the central waypoint

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])
ax.scatter(glider_av_df.x/1000, glider_av_df.y/1000)
ax.set(xlim=[-8, 8], ylim=[-8, 8], xlabel='Zonal distance from central waypoint (km)', ylabel='Meridional distance from central waypoint (km)')
savefig('dive_locations')

As above for the density of glider dives in each 1 km square box

In [ ]:
fig, ax = plt.subplots(figsize=[12,10])
c = ax.pcolor(lon_edges/1000, lat_edges/1000,dives_in, cmap='Blues')
ax.set(xlabel='Zonal distance from center (km)', ylabel='Meridional distance from center (km)')
fig.colorbar(c, ax=ax, label='Number of dives in 1km square box')
savefig('Dive_concentration')


The maximum distance between two glider dives is 15 km. At 14 degrees north of the equator in a water depth of several km, the approximation of a virtual mooring seems appropriate. 

Plotting the variation of dive average current during the deployment. These are coloured by heading. 

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(16,14), sharex=True)
ax = ax.ravel()
ax[0].scatter(glider_av_df.glider_time, glider_av_df.DAC_u, c=glider_av_df.heading)
scat = ax[1].scatter(glider_av_df.glider_time, glider_av_df.DAC_v, c=glider_av_df.heading)
fig.autofmt_xdate()

ax[0].set(ylabel='DAC u $(\mathrm{m\ s^{-1}})$', ylim=[-0.2, 0.2])
ax[1].set(ylabel='DAC v $(\mathrm{m\ s^{-1}})$', xlim = [glider_av_df.glider_time[0]- timedelta(days=1), glider_av_df.glider_time.iloc[-1]+ timedelta(days=1)], ylim=[-0.2, 0.2])
sub_ax = plt.axes([0.62, 0.82, 0.2, 0.02])
fig.colorbar(mappable=scat, orientation='horizontal',cax=sub_ax, label='glider heading')
savefig('DAC_by_dive_bearing')

Two patterns are apparent, the long term change in current speed and direction and a short term signal, more obvious in the u component, that shows a dependency on heading. This is a sign of a poorly calibrated compass

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(x, y)
    deg = (180/np.pi)*phi
    if deg<0:
        deg = 360+deg
    return(rho, deg)
cart2pol=np.vectorize(cart2pol)

In [ ]:
glider_av_df['current_speed'], glider_av_df['current_bearing'] = cart2pol(glider_av_df.DAC_u, glider_av_df.DAC_v)

Plotting current speed and direction shows a switch around 5th February. There is nothing in the glider logs around this time. The compass calibration was performed on January 29.

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(16,14), sharex=True)
ax = ax.ravel()
ax[0].scatter(glider_av_df.glider_time, glider_av_df.current_speed, c=glider_av_df.heading, vmin=0, vmax=360)
scat = ax[1].scatter(glider_av_df.glider_time, glider_av_df.current_bearing, c=glider_av_df.heading, vmin=0, vmax=360)
ax[0].set(ylabel='Current speed $(\mathrm{m\ s^{-1}})$', ylim=[0, 0.3])
ax[1].set(ylabel='Current bearing $(\mathrm{m\ s^{-1}})$', xlim = [glider_av_df.glider_time[0]- timedelta(days=1), glider_av_df.glider_time.iloc[-1]+ timedelta(days=1)], ylim=[0,360])
sub_ax = plt.axes([0.62, 0.22, 0.2, 0.02])
fig.colorbar(mappable=scat, orientation='horizontal',cax=sub_ax, label='glider heading')
#savefig('current_heading')

In [ ]:
fig, ax = plt.subplots( figsize=(10,10))
ax.scatter(glider_df.abs_salinity, glider_df.temp, c=glider_df.glider_time, s = 1)
ax.set(xlabel='Absolute salinity', ylabel='Conserative temperature')
savefig('tempsal')


Make a new data frame, averaging in depth bins

In [ ]:
z_bin_size = 5
z_centers = np.arange(-800+z_bin_size/2, 0, z_bin_size)
z_centers
noot = []
for depth in glider_df.depth:
    noot.append(np.abs(z_centers+depth).argmin())
   
glider_df['depth_bin']= noot

In [ ]:
glider_z_av_df.abs_salinity.min()

In [ ]:
glider_z_av_df = glider_df.groupby('depth_bin', as_index=False).median()

In [ ]:
glider_df['sound_speed'] = gsw.sound_speed(glider_df.abs_salinity, glider_df.temp, glider_df.pressure)
fix_sal = np.empty((len(glider_z_av_df.index)))
fix_sal[:] = 35
sound_speed_adcp = gsw.sound_speed(fix_sal, glider_z_av_df.temp, glider_z_av_df.pressure)
fix_sal[:] = glider_z_av_df.abs_salinity.min()
sound_speed_min_sal = gsw.sound_speed(fix_sal, glider_z_av_df.temp, glider_z_av_df.pressure)
fix_sal[:] = glider_z_av_df.abs_salinity.max()
sound_speed_max_sal = gsw.sound_speed(fix_sal, glider_z_av_df.temp, glider_z_av_df.pressure)


In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
ax.plot(glider_z_av_df.sound_speed, -glider_z_av_df.depth, label='From glider measurements')
ax.plot(sound_speed_adcp, -glider_z_av_df.depth, label='Used by ADCP')
#ax.plot(sound_speed_min_sal, -glider_z_av_df.depth, label='Calculted with minimum observed salinity')
#ax.plot(sound_speed_max_sal, -glider_z_av_df.depth, label='Calculted with maximum oberved salinity')
ax.set(xlabel='Speed of sound (m/s)', ylabel='z (m)', ylim=[-760, 0])
ax.legend()
savefig('speed_of_sound')

In [ ]:
glider_z_av_df['sound_speed_diff'] = np.abs(glider_z_av_df.sound_speed - sound_speed_adcp)
glider_z_av_df['sound_speed_diff'].describe()

### Difference between sound speed used by ADCP and observed in the water
Average difference of 0.9 m/s (0.06%) with a maximum difference of 2.9 m/s (0.2 %)

Taking these differnces as small realtive to the absolute values of sound speed observed and the accuracy of the ADCP, we consider this error in sound speed to be negligible

### importing the glider surfacings for drift calcualtion

In [ ]:
foo = scipy.io.loadmat('/media/callum/storage/Documents/foo/hero.mat')['proc_with_att']

In [ ]:
gps_post_pre_dive = scipy.io.loadmat('/media/callum/storage/Documents/foo/gpspostpredive.mat')['foo']
gps_post_dive = scipy.io.loadmat('/media/callum/storage/Documents/foo/gpspostdive.mat')['foo']
gps_pre_dive = scipy.io.loadmat('/media/callum/storage/Documents/foo/gpspredive.mat')['foo']

In [ ]:
gps_dives_df = pd.DataFrame({'lon_post_pre':gps_post_pre_dive[:,1], 'lat_post_pre':gps_post_pre_dive[:,0], 'time_post_pre':glidertimetoneat(gps_post_pre_dive[:,2])[1], 'lon_pre':gps_pre_dive[:,1], 'lat_pre':gps_pre_dive[:,0], 'time_pre':glidertimetoneat(gps_pre_dive[:,2])[1], 'lon_post':gps_post_dive[:,1], 'lat_post':gps_post_dive[:,0], 'time_post':glidertimetoneat(gps_post_dive[:,2])[1]}, index=None)

Calculate the drift time fro eac surfacing and, from that, the surface drift velocity

In [ ]:
time_drift = gps_dives_df_dt.time_pre - gps_dives_df_dt.time_post_pre
time_list = []
for time_point in time_drift:
    time_list.append(time_point.seconds)
gps_dives_df['drift_seconds'] = time_list

In [ ]:
gps_dives_df['drift_u'] = ((gps_dives_df.lon_pre-gps_dives_df.lon_post_pre)* deg_lat * np.cos(np.deg2rad(origin[1])))/gps_dives_df.drift_seconds
gps_dives_df['drift_v'] = ((gps_dives_df.lat_pre-gps_dives_df.lat_post_pre)* deg_lat)/gps_dives_df.drift_seconds

In [ ]:
gps_dives_df.describe()

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,figsize=(14,12))
ax = ax.ravel()
ax[0].scatter(gps_dives_df.time_post_pre,gps_dives_df.drift_u)
ax[1].scatter(gps_dives_df.time_post_pre, gps_dives_df.drift_v)
#ax[0].scatter(gps_dives_df.time_pre,gps_dives_df.lon_pre)
ax[0].set(xlim=[datetime(2020,1,23), datetime(2020,2,18)], ylim=[-0.5, 0.5],ylabel='Zonal drift speed (m/s)')
ax[1].set(ylim=[-0.5, 0.5],ylabel='Meridional drift speed (m/s)')
fig.autofmt_xdate()
savefig('surface_drift_speed')

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,figsize=(14,12))
ax = ax.ravel()
ax[0].scatter(gps_dives_df.time_post_pre,(gps_dives_df.lon_pre-gps_dives_df.lon_post_pre)* deg_lat * np.cos(np.deg2rad(origin[1])))
ax[1].scatter(gps_dives_df.time_post_pre,(gps_dives_df.lat_pre-gps_dives_df.lat_post_pre)* deg_lat)
#ax[0].scatter(gps_dives_df.time_pre,gps_dives_df.lon_pre)
ax[0].set(xlim=[datetime(2020,1,23), datetime(2020,2,18)], ylim=[-300, 300],ylabel='Zonal drift distance (m)')
ax[1].set(ylim=[-300, 300],ylabel='Meridional drift distance (m)')
fig.autofmt_xdate()
savefig('surface_drift_distance')

In [ ]:
fig,ax = plt.subplots(figsize=(14,12))
ax.quiver(gps_dives_df.lon_pre, gps_dives_df.lat_pre, gps_dives_df.drift_u, gps_dives_df.drift_v)
savefig('surface_drift_bowtie')